# Análise de Portfólio de Ativos
Análise exploratória e indicadores de retorno, risco e performance de um portfólio de ativos financeiros.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

- Dados para Leitura

In [ ]:
Acoes = ["ITUB4.SA", "CPLE3.SA", "LREN3.SA", "EMBR3.SA", "PSSA3.SA", "GOAU4.SA"]
Periodo_Inicio = datetime.datetime(2016,1,1)
Periodo_Termino = datetime.datetime(2021,2,10)

- Dados do Portfolio

In [ ]:
PortfolioDistribuicao = [0.10, 0.20, 0.20, 0.10, 0.10, 0.30]
Portfolio = pd.DataFrame(data={"Acoes":Acoes, "Pesos":PortfolioDistribuicao}).set_index("Acoes")
Portfolio

In [ ]:
DadosAcoes = pdr.get_data_yahoo(Acoes, start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados = DadosAcoes["Adj Close"]

In [ ]:
Dados

### Análise descritiva dos Preços
- Contagem de dias.
- Média dos preços do ativo.
- Desvio padrão dos preços.
- Preço Mínimo.
- Preço Máximo.
- Quartis dos Preços.

In [ ]:
Dados.describe()

In [ ]:
SeriePrecos = plg.Figure()
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["CPLE3.SA"], name="Copel", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["EMBR3.SA"], name="Embraer", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["GOAU4.SA"], name="Gerdau", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["ITUB4.SA"], name="Itau Holding", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["LREN3.SA"], name="Renner", mode="lines"))
SeriePrecos.add_trace(plg.Scatter(x=Dados.index, y=Dados["PSSA3.SA"], name="Porto Seguro", mode="lines"))
SeriePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(SeriePrecos, "Preços")

### Cálculo e análise descritiva dos Retornos
- Retorno Médio do Ativo (Média).
- Risco do Ativo (Desvio Padrão).
- Maior perda do período (Mínimo).
- Maior ganho no período (Máximo).
- Quartis do Risco.

In [ ]:
RetornoDosAtivos = Dados.pct_change().fillna(0)

In [ ]:
RetornoDosAtivos

In [ ]:
RetornoDosAtivos.describe()

In [ ]:
SerieRetornos = plg.Figure()
SerieRetornos.add_trace(plg.Scatter(x=RetornoDosAtivos.index, y=RetornoDosAtivos["CPLE3.SA"], name="Copel", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=RetornoDosAtivos.index, y=RetornoDosAtivos["EMBR3.SA"], name="Embraer", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=RetornoDosAtivos.index, y=RetornoDosAtivos["GOAU4.SA"], name="Gerdau", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=RetornoDosAtivos.index, y=RetornoDosAtivos["ITUB4.SA"], name="Itau Holding", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=RetornoDosAtivos.index, y=RetornoDosAtivos["LREN3.SA"], name="Renner", mode="lines"))
SerieRetornos.add_trace(plg.Scatter(x=RetornoDosAtivos.index, y=RetornoDosAtivos["PSSA3.SA"], name="Porto Seguro", mode="lines"))
SerieRetornos.update_layout(yaxis_tickformat=".1%")
Plot_Show(SerieRetornos, "Retornos por Ativo")

- Análise Risco vs Retorno dos Ativos

In [ ]:
DadosRiscoRetorno = RetornoDosAtivos.describe().transpose()

In [ ]:
DadosRiscoRetorno.columns = ["Dias", "RetornoMedio", "Risco", "Minimo", "p25", "p50", "p75", "Maximo"]

In [ ]:
DadosRiscoRetorno

In [ ]:
RiscoRetorno = plg.Figure(plx.scatter(DadosRiscoRetorno, x="Risco", y="RetornoMedio", text=DadosRiscoRetorno.index))
RiscoRetorno.update_layout(xaxis_tickformat=".2%", yaxis_tickformat=".2%")
RiscoRetorno.update_traces(textposition="top center")
Plot_Show(RiscoRetorno, "Posicionamento dos Ativos")

### Retorno Acumulado por Ativo
Calcula dia-a-dia o retorno acumulado de cada ativo, sem ponderação da distribuição do portfólio, apenas usando o período.

In [ ]:
RetornoAcumuladoDosAtivos = (1+RetornoDosAtivos).cumprod()-1

In [ ]:
RetornoAcumuladoDosAtivos

Retorno Acumulado de cada ativo

In [ ]:
RetornoAcumuladoDosAtivos[-1:]

In [ ]:
SerieRetornosAcum = plg.Figure()
SerieRetornosAcum.add_trace(plg.Scatter(x=RetornoAcumuladoDosAtivos.index, y=RetornoAcumuladoDosAtivos["CPLE3.SA"], name="Copel", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=RetornoAcumuladoDosAtivos.index, y=RetornoAcumuladoDosAtivos["EMBR3.SA"], name="Embraer", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=RetornoAcumuladoDosAtivos.index, y=RetornoAcumuladoDosAtivos["GOAU4.SA"], name="Gerdau", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=RetornoAcumuladoDosAtivos.index, y=RetornoAcumuladoDosAtivos["ITUB4.SA"], name="Itau Holding", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=RetornoAcumuladoDosAtivos.index, y=RetornoAcumuladoDosAtivos["LREN3.SA"], name="Renner", mode="lines"))
SerieRetornosAcum.add_trace(plg.Scatter(x=RetornoAcumuladoDosAtivos.index, y=RetornoAcumuladoDosAtivos["PSSA3.SA"], name="Porto Seguro", mode="lines"))
SerieRetornosAcum.update_layout(yaxis_tickformat=".1%")
Plot_Show(SerieRetornosAcum, "Retornos Acumulados por Ativos")

### Indicadores de cada Ativo

- Retorno Médio (%)

In [ ]:
RetornoMedioDosAtivos = RetornoDosAtivos.mean()
np.round(RetornoMedioDosAtivos * 100, 2)

- Retorno Acumulado

In [ ]:
RetornoAcumulado = RetornoAcumuladoDosAtivos[-1:].transpose()
RetornoAcumulado.columns = ["Last"]

In [ ]:
SerieRetornoAcumulado = plg.Figure(plx.bar(RetornoAcumulado, x=RetornoAcumulado.index, y="Last", text="Last"))
SerieRetornoAcumulado.update_layout(yaxis_tickformat=".1%")
SerieRetornoAcumulado.update_traces(texttemplate='%{text:.2%}', textposition='outside')
Plot_Show(SerieRetornoAcumulado, "Retorno Acumulado por Ativo")

- Risco (%)

In [ ]:
RiscoDosAtivos = RetornoDosAtivos.std()
np.round(RiscoDosAtivos * 100, 2)

- Performance (Sharpe %)

In [ ]:
SharpeDosAtivos = RetornoMedioDosAtivos / RiscoDosAtivos
np.round(SharpeDosAtivos * 100, 2)

### Retorno Acumulado do Portfólio
Calcula dia-a-dia os retornos e retorno acumulado do portfólio, conforme distribuição nos ativos.

- Retorno dia-a-dia do portfólio

In [ ]:
RetornosDoPortfolio = Portfolio.transpose().dot(RetornoDosAtivos.transpose()).transpose()

In [ ]:
RetornoDosAtivos["Portfolio"] = RetornosDoPortfolio["Pesos"]

In [ ]:
RetornoDosAtivos

In [ ]:
SerieRetornos = plg.Figure(plg.Scatter(x=RetornoDosAtivos.index, y=RetornoDosAtivos["Portfolio"], name="Portfólio", mode="lines"))
SerieRetornos.update_layout(yaxis_tickformat=".1%")
Plot_Show(SerieRetornos, "Retorno do Portfólio")

In [ ]:
HistogramaRetornos = plg.Figure(plg.Histogram(x=RetornoDosAtivos["Portfolio"]))
HistogramaRetornos.update_layout(xaxis_tickformat=".1%")
Plot_Show(HistogramaRetornos, "Retornos do Portfólio")

- Retorno acumulado do portfólio

In [ ]:
RetornoDosAtivos["Acumulado"] = (1+RetornoDosAtivos["Portfolio"]).cumprod()-1

In [ ]:
RetornoDosAtivos

In [ ]:
RetornoDosAtivos[-1:]

In [ ]:
SerieRetornos = plg.Figure(plg.Scatter(x=RetornoDosAtivos.index, y=RetornoDosAtivos["Acumulado"], name="Portfólio", mode="lines"))
SerieRetornos.update_layout(yaxis_tickformat=".1%")
Plot_Show(SerieRetornos, "Retorno Acumulado do Portfólio")

### Indicadores do Portfólio

- Retorno Médio

In [ ]:
RetornoMedio = RetornoDosAtivos["Portfolio"].mean()
print("Retorno Médio: {0:.2%}".format(RetornoMedio))

- Retorno Acumulado

In [ ]:
RetornoAcumulado = RetornoDosAtivos["Acumulado"][-1]
print("Retorno Acumulado: {0:.2%}".format(RetornoAcumulado))

- Risco

In [ ]:
Risco = RetornoDosAtivos["Portfolio"].std()
print("Risco: {0:.2%}".format(Risco))

- Performance (Sharpe)

In [ ]:
Sharpe = RetornoMedio / Risco
print("Performance Sharpe: {0:.2%}".format(Sharpe))

- VaR Histórico

In [ ]:
InvestimentoPrevisto = 10000

In [ ]:
VarHistorico = np.quantile(RetornoDosAtivos["Portfolio"], 0.05)
VarHistoricoPeriodos = np.sqrt([1,7,14,21]) * VarHistorico

PerdaMaxima = InvestimentoPrevisto * VarHistoricoPeriodos

print("VaR 1 dia: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[0], PerdaMaxima[0]))
print("VaR 7 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[1], PerdaMaxima[1]))
print("VaR 14 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[2], PerdaMaxima[2]))
print("VaR 21 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarHistoricoPeriodos[3], PerdaMaxima[3]))

- VaR Paramétrico

In [ ]:
VarParam = RetornoMedio - norm.ppf(0.95) * Risco
VarParamPeriodos = np.sqrt([1,7,14,21]) * VarParam

PerdaMaxima = InvestimentoPrevisto * VarParamPeriodos

print("VaR 1 dia: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[0], PerdaMaxima[0]))
print("VaR 7 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[1], PerdaMaxima[1]))
print("VaR 14 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[2], PerdaMaxima[2]))
print("VaR 21 dias: {0:.2%}, Perda máxima de {1:.2f}".format(VarParamPeriodos[3], PerdaMaxima[3]))